#### 1.Bibliothèques 

In [6]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score

#### 2. Chargement des données des attributs pertinents (4 sur 7 attributs seulement age, bmi, childre, charges) depuis le fichier 'insurance.csv' 
#### D'aprés l'analyse du dataset de régression (étape 4)

In [7]:
data = pd.read_csv('insurance.csv')
data.columns
data = data[['age','bmi', 'children', 'charges']]
print (data.shape)
print (data.columns)

(1338, 4)
Index(['age', 'bmi', 'children', 'charges'], dtype='object')


#### 3.  Correction de données manquantes : exemple de complétion de ces valeurs par la moyenne de la colonne
PS : Nous n'avons pas de valeurs manquantes donc ce bout de code ne va pas influencer notre code ou données  

In [8]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data)
data=imp.transform(data)

#### 4. Séprataion des attributs (Inputs) et de l'attribut objectif (Output)

In [9]:
NB_FEATURES=3
X=data[:,0:NB_FEATURES]         # Features: inputs (21 premières colonnes)
y=data[:,NB_FEATURES:].ravel()  # Target : output/ (Dernière colonne)

#### 5. Découpage de l'ensemble de données : 80% TRAINING, 20% TEST

In [10]:
TEST_SIZE = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,shuffle=True)

#### 6. Affihage des quelques informations concernant notre dataset 

In [11]:
print(f" >>> Taille du dataset / nombre d'instances :\t {data.shape[0]} ")
print(f" >>> Nombre d'attributs / Inputs : \t\t {NB_FEATURES} ")
print(f" >>> Taille du dataset d'entrainement : \t {X_train.shape[0]}")
print(f" >>> Taille du dataset de test : \t\t {X_test.shape[0]}")

 >>> Taille du dataset / nombre d'instances :	 1338 
 >>> Nombre d'attributs / Inputs : 		 3 
 >>> Taille du dataset d'entrainement : 	 1070
 >>> Taille du dataset de test : 		 268


#### 7. Normalisation des données

In [12]:
scaler=preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# Etape 2 : Paramètres Par Défaut
#### 8. Création d'un modèle de regression avec les paramètres par défaut de ScikitLearn + apprentissage du modèle

In [13]:
mlp =MLPRegressor(hidden_layer_sizes=(100,), activation='relu',  solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
                   power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, 
                   )

mlp.fit(X_train,y_train)
y_pred=mlp.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


#### 9. Métrique de régréssion

In [14]:
print(" Score final %.2f" % r2_score(y_test, y_pred))

 Score final -1.10


# Etape 3 : Paramètres Optimaux (Meilleur Modèle - GridSearchCV)

#### 10. Utilisation de la méthode GridSearchCV qui trouve facilement les valeurs optimales parmi les valeurs données pour les paramètres pertinents (activation, solver, learning_rate, max_iter)

In [15]:
from sklearn.model_selection import GridSearchCV

mlp_gs = MLPRegressor(max_iter=3000)
parameter_space = {
    'activation': ['relu', 'identity', 'logistic', 'tanh'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'learning_rate': ['constant','adaptive', 'invscaling'],
    'max_iter': [200, 2000, 3000],
}

#n_jobs=-1 , -1 sert à utiliser tous les cœurs de processeur disponibles.
#cv = 10 est pour la validation croisée, ici cela signifie 10 fois la validation croisée stratifiée (K-fold)
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=10)

#Apprentissage / Entrainement de modèle
clf.fit(X, y)

#Affichage des scores pour toutes les combinaisons (modèles) possibles 
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

i=0
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    i=i+1    
print("Le nombre de combinaisons (modèles) possible est : %r " % (i)) 
print(" \n ")
print("|  Mean  |   Std   |  Combinaison / Modèle ")
print("-------------------------------------------------------------------------------------------------------------------")

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f    | %0.03f   | %r" % (mean, std, params))

Le nombre de combinaisons (modèles) possible est : 108 
 
 
|  Mean  |   Std   |  Combinaison / Modèle 
-------------------------------------------------------------------------------------------------------------------
0.055    | 0.038   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'adam'}
0.108    | 0.037   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'lbfgs'}
-154286905.709    | 439996617.229   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'sgd'}
0.102    | 0.032   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 2000, 'solver': 'adam'}
0.098    | 0.041   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 2000, 'solver': 'lbfgs'}
-1920786519.841    | 5743449428.392   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 2000, 'solver': 'sgd'}
0.103    | 0.033   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 3000, 's

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [ 5.54065168e-02  1.08113649e-01 -1.54286906e+08  1.02271044e-01
  9.83635539e-02 -1.92078652e+09  1.02800934e-01  1.05118747e-01
 -8.49263344e+08  4.94732863e-02  1.04526464e-01 -5.54033293e+07
  1.03242647e-01  8.20210510e-02 -9.77643831e+07  1.02279121e-01
  1.00998543e-01 -4.47981086e+07  5.06230938e-02  1.07402022e-01
 -1.71670070e+08  1.02957199e-01  1.05009541e-01 -2.03756337e+09
  1.01901736e-01  6.83751151e-02 -3.18909937e+08  1.02085578e-01
  1.11310112e-01             nan  1.03305675e-01  1.11310399e-01
             nan  1.03447778e-01  1.11310057e-01             nan
  1.02783820e-01  1.11310155e-01             nan  1.02415056e-01
  1.11309645e-01             nan  1.02218817e-01  1.11310736e-01
             nan  1.01928115e-01  1.11309457e-01             nan
  1.01979411e-01  1.11310055e-01             nan  1.02947635e-01
  1.11309960e-01

#### 11. Affichage du meilleur modèle (hyperparamètres optimaux parmi les valeurs données). 

In [16]:
print('Meilleurs paramètres trouvés : \n', clf.best_params_)
print('Meilleurs paramètres trouvés : \n', clf.best_estimator_)

Meilleurs paramètres trouvés : 
 {'activation': 'identity', 'learning_rate': 'adaptive', 'max_iter': 3000, 'solver': 'lbfgs'}
Meilleurs paramètres trouvés : 
 MLPRegressor(activation='identity', learning_rate='adaptive', max_iter=3000,
             solver='lbfgs')


#### 12. Métrique de régression (sur le meilleur modèle obtenu)

In [19]:
print ("Score final : ", round(clf.score(X, y),2))

Score final :  0.12
